In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
from keras import metrics
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import preprocessing
X, y, class_names = preprocessing.create_data_tensor()

/home/parth/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/parth/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/parth/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:2233: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted a

In [3]:
print('Dimensions of X:', X.shape)
print('Possible Classes:', class_names.values())

Dimensions of X: (2565, 22, 57)
Possible Classes: dict_values(['make', 'polite', 'draw', 'soon', 'money', 'cost', 'when', 'innocent', 'pen', 'name', 'know', 'paper', 'no', 'I', 'tray', 'research', 'computer_PC_', 'ready', 'God', 'what', 'wait_notyet_', 'building', 'yes', 'different', 'sad', 'man', 'right', 'later', 'all', 'hurry', 'his_hers', 'hear', 'danger', 'eat', 'drink', 'share', 'thank', 'you', 'temper', 'juice', 'hurt', 'wild', 'please', 'give', 'come', 'glove', 'forget', 'more', 'which', 'shop', 'lose', 'maybe', 'stubborn', 'question', 'where', 'sorry', 'spend', 'girl', 'Norway', 'write', 'science', 'zero', 'buy', 'happy', 'hot', 'not', 'take', 'will', 'head', 'go', 'is_true_', 'think', 'why', 'deaf', 'answer', 'surprise', 'how', 'read', 'love', 'flash', 'boy', 'voluntary', 'hello', 'cold', 'change_mind_', 'mine', 'crazy', 'responsible', 'who', 'joke', 'same', 'wrong', 'alive', 'us', 'exit'])


## Changing the number of classes

In [4]:
import numpy as np
NUM_CLASSES = 20
f = np.isin(y, [i for i in range(NUM_CLASSES)])
len(y[f])
y = y[f]
X = X[f]

In [5]:
X_train_temp, X_test_temp, y_train, y_test = train_test_split(X, y, test_size=0.3)
print('Dimensions of X_train: ', X_train_temp.shape)
print('Dimensions of X_test: ', X_test_temp.shape)

Dimensions of X_train:  (378, 22, 57)
Dimensions of X_test:  (162, 22, 57)


## Changing the dimensions of the input matrix

In [6]:
X_train = X_train_temp.swapaxes(1, 2)
X_test = X_test_temp.swapaxes(1, 2)
print('Dimensions after swapping the feature axis with the timesteps axis')
print('Dimensions of X_train:', X_train.shape)
print('Dimensions of X_test:', X_test.shape)

Dimensions after swapping the feature axis with the timesteps axis
Dimensions of X_train: (378, 57, 22)
Dimensions of X_test: (162, 57, 22)


## Two different LSTM models

In [7]:
lstm_model = Sequential()

# Model 1
lstm_model.add(LSTM(units=32, input_shape=(57, 22), return_sequences=True))
lstm_model.add(LSTM(units=32, return_sequences=True))
lstm_model.add(LSTM(units=300))

# Model 2
# lstm_model.add(LSTM(units=500, input_shape=(57, 22), return_sequences=True))
# lstm_model.add(LSTM(units=500))
# lstm_model.add(Dropout(0.5))

lstm_model.add(Dense(NUM_CLASSES, activation='tanh'))
# lstm_model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])
lstm_model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


## Training

In [8]:
from keras.utils import to_categorical
lstm_model.fit(X_train, to_categorical(y_train), batch_size=16, epochs=30)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
378/378 [==============================] - 10s 26ms/step - loss: 0.0487 - acc: 0.0397
Epoch 2/30
378/378 [==============================] - 7s 20ms/step - loss: 0.0479 - acc: 0.0529
Epoch 3/30
378/378 [==============================] - 7s 20ms/step - loss: 0.0477 - acc: 0.0714
Epoch 4/30
378/378 [==============================] - 7s 20ms/step - loss: 0.0479 - acc: 0.0635
Epoch 5/30
378/378 [==============================] - 8s 20ms/step - loss: 0.0469 - acc: 0.0688
Epoch 6/30
378/378 [==============================] - 8s 20ms/step - loss: 0.0459 - acc: 0.1243
Epoch 7/30
378/378 [==============================] - 8s 20ms/step - loss: 0.0452 - acc: 0.1825
Epoch 8/30
378/378 [==============================] - 7s 20ms/step - loss: 0.0446 - acc: 0.1958
Epoch 9/30
378/378 [==============================] - 8s 20ms/step - loss: 0.0431 - acc: 0.2487
Epoch 10/30
378/378 [==============================] - 8s 20ms/step - loss: 0.0416 - ac

## Evaluation

In [9]:
score = lstm_model.evaluate(X_test, to_categorical(y_test), batch_size=8)
print(score)

162/162 [==============================] - 2s 11ms/step
[0.017735756651992785, 0.7407407407407407]
